## Exploration base de données

In [1]:
import pandas as pd
import os
from datetime import date, timedelta

In [2]:
# racine des fichiers quotidiens
base_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{}.csv'

#dates de disponibilité des fichiers
start_date = date(2020, 1, 22)   # start date
end_date = date(2020, 3, 13)   # end date

#répertoire de sauvegarde des fichiers bruts
rawfiles_dir = '../data/raw/'
processed_dir = '../data/processed/'

# Boucle de récupération des fichiers

In [3]:
delta = end_date - start_date

for i in range(delta.days + 1):
    day = start_date + timedelta(days=i)
    day_label = day.strftime("%m-%d-%Y")
    virus_df = pd.read_csv(base_url.format(day_label), sep=',', parse_dates=['Last Update'])
    virus_df.to_csv(os.path.join(rawfiles_dir, day_label + '.csv'), index=False)

In [4]:
pd.read_csv(rawfiles_dir + '03-05-2020.csv', sep=',')

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude
0,Hubei,Mainland China,2020-03-05 14:53:03,67466,2902,40592,30.9756,112.2707
1,NaN,South Korea,2020-03-05 09:03:09,6088,35,41,36.0000,128.0000
2,NaN,Italy,2020-03-05 17:43:03,3858,148,414,43.0000,12.0000
3,NaN,Iran,2020-03-05 13:43:04,3513,107,739,32.0000,53.0000
4,Guangdong,Mainland China,2020-03-05 09:23:03,1351,7,1181,23.3417,113.4244
...,...,...,...,...,...,...,...,...
168,NaN,Ukraine,2020-03-03 15:33:02,1,0,0,48.3794,31.1656
169,From Diamond Princess,Australia,2020-02-29 02:03:10,0,0,0,35.4437,139.6380
170,"Lackland, TX (From Diamond Princess)",US,2020-02-24 23:33:02,0,0,0,29.3829,-98.6134
171,"Omaha, NE (From Diamond Princess)",US,2020-02-24 23:33:02,0,0,0,41.2545,-95.9758


## Constitution de la table de réference lat / log

In [5]:
import glob

df_list = []

for file in glob.glob(os.path.join(rawfiles_dir, '*.csv')):
    virus_df = pd.read_csv(file, sep=',')
    if 'Latitude' in virus_df.columns and 'Longitude' in virus_df.columns:
        df_list.append(virus_df)
        
all_df = pd.concat(df_list)

(all_df[['Province/State', 'Country/Region', 'Latitude', 'Longitude']]
 .drop_duplicates(subset=['Province/State', 'Country/Region'])
 .sort_values(by=['Country/Region', 'Province/State'])
.to_csv(os.path.join(processed_dir, 'lat_long_table.csv'), index=False)
 
)

## Construction d'une table unique

In [6]:
data_catalog = {
    'Last Update': ['<M8[ns]'],
    'Confirmed' : ['float','int64'],
    'Deaths':['float','int64'],
    'Recovered':['float','int64'],
    'Latitude': ['float64'],
    'Longitude':['float64'],
}

In [7]:
df_list=[]
latlong_df = pd.read_csv(os.path.join(processed_dir, 'lat_long_table.csv'))

# Lecture des fichiers récupérés et sélection de
for file in glob.glob(os.path.join(rawfiles_dir, '*.csv')):
    virus_df = pd.read_csv(file, sep=',',parse_dates=['Last Update'])
    if not ('Latitude' in virus_df.columns or 'Longitude' in virus_df.columns):
        virus_df = virus_df.merge(latlong_df, on=['Province/State','Country/Region'], how='left')
    
    for field, types in data_catalog.items():
        assert virus_df[field].dtypes in types, f"Bad for {field} in {file}"
   
    df_list.append(virus_df.assign(source=os.path.basename(file)))
                        

all_df =  pd.concat(df_list)
                        
# Sauvegarde de la base finale
all_df.to_csv(os.path.join(processed_dir, 'all_data.csv'), index = False)

In [8]:
all_df.dtypes

Province/State            object
Country/Region            object
Last Update       datetime64[ns]
Confirmed                float64
Deaths                   float64
Recovered                float64
Latitude                 float64
Longitude                float64
source                    object
dtype: object

In [9]:
all_df.shape

(5383, 9)